In [167]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
driver = webdriver.Chrome(executable_path='C:\chromedriver\chromedriver.exe')


In [159]:
##HELPERFUNCTIONS

def get_matchLinks(page):
    matchesLinks = []
    ## https://www.hltv.org/results?offset=0
    driver.get('https://www.hltv.org/results?offset='+ str(page) +'&stars=1')
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    ## FIND MATCHES
    for div in soup.findAll('div', attrs={'class': 'results'}):
        for a in div.findAll('a', attrs={'class': 'a-reset'}):
            link = a['href']
            if (link[:8] == "/matches"):
                matchesLinks.append(link)
    return matchesLinks

def get_page_data(link):
    url = 'https://www.hltv.org/' + str(link)
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)    
    return soup

def get_game_links(soup):
    statlinks = []
    for div in soup.findAll('div', attrs={'class': 'results-center-stats'}):
        for a in div.findAll('a'):
            statlinks.append(a['href'])
    return statlinks

def get_game_stats(link):
    result = get_page_data(link)
    return result

In [160]:
## GAME STAT INFO
##GATHER PLAYER STATS

#print(data[8][0])
#LINK /stats/matches/mapstatsid/118030/astralis-vs-og
def gather_player_stats(link):
    colnames = ["Player","K (hs)","A (f)","D","KAST","K-D Diff","ADR","FK Diff","Rating","Team"]

    #localLink = data[8][0]
    localLink = link
    result = get_game_stats(localLink)
    result = result.findAll('table', {'class':'stats-table'})
    page_source = result
    results = pd.read_html(str(result))

    MapID = localLink.split("/")[4]

    #print(MapID)
    #print(Map)
    df_team1 = pd.DataFrame(results[0])
    df_team1['Team'] = df_team1.columns[0]
    df_team1.columns = colnames
    #df_team1["Player"].replace('\r\n',regex=True,inplace=True)

    df_team2 = pd.DataFrame(results[1])
    df_team2['Team'] = df_team2.columns[0]
    df_team2.columns = colnames
    #df_team2["Player"].replace('\r\n',regex=True,inplace=True)

    game_df = df_team1.append(df_team2,ignore_index=True)
    game_df['MapID'] = MapID
    #game_df['Map'] 
    #print(game_df)
    #game_df.to_csv(index=True)
    return game_df


In [161]:
##BASIC INFO

def gather_match_stats(links):
    match_df = pd.DataFrame()
    ##GET DATE from https://www.hltv.org/matches/2347834***
    names = ["Date","Tournament","Team1","Result1","Team2","Result2","Map","MapID","StatLinks"]
    #names = ["Date","Tournament","Team1","Result1","Team2","Result2"]
    data = []
    maps = []
    mapsID = []
    players = []

    for div in soup.findAll('div', attrs={'class': 'match-page'}):
        pageDate = div.find('div', attrs={'class': 'date'})
        pageTournament = div.find('div', attrs={'class': 'event text-ellipsis'})
        data.append(pageDate.text)
        data.append(pageTournament.text)

    ##GET FINALRESULT1 from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'team1-gradient'}):
        pageTeam1 = div.find('div', attrs={'class': 'teamName'})
        pageResult1 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
        data.append(pageTeam1.text)
        data.append(pageResult1.text)

    ##GET FINALRESULT2 from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'team2-gradient'}):
        pageTeam2 = div.find('div', attrs={'class': 'teamName'})
        pageResult2 = div.find('div', attrs={'class': ['won', 'lost', 'tie']})
        data.append(pageTeam2.text)
        data.append(pageResult2.text)

    ##GET MAPS AND MAPSID from https://www.hltv.org/matches/2347834***
    for div in soup.findAll('div', attrs={'class': 'stats-menu-link'}):
        map1 = div.findAll('div', attrs= {'class': 'dynamic-map-name-full'})
        for div in map1:
            maps.append(div.text)
            mapsID.append(div['id'])

    #map_df = pd.DataFrame(maps)
    data.append(maps)
    #mapsID_df = pd.DataFrame(mapsID)
    data.append(mapsID)


    ##GET DETAILED STATS LINKS
    data.append(links)

    ##GET PLAYER INFO
    def get_player_info_def:
        for div in soup.findAll('div', attrs={'class': 'lineups'}):
            for table in div.findAll('td',attrs={'class': 'player player-image'}):
                for a in table.findAll('a'):
                    players.append(a['href'])
        player_df = pd.DataFrame(players)
        data_dict = {'data':data}
        data_df = pd.DataFrame(data_dict)

    #data_df.index = names
    #-----------
    #maps_df = pd.DataFrame(data_df.iloc[[7]].values)
    #print(data_df.iloc[[7]].values)
    #print(data_df)
    for i in range(1,len(maps)):
        #print(maps[i])
        #print(mapsID[i])
        #print(links[i-1])
        game_df = gather_player_stats(links[i-1])
        game_df['Map'] = maps[i]
        game_df['Tournament'] = data[1]
        game_df['Date'] = data[0]
        game_df['Team1'] = data[2]
        game_df['Team2'] = data[4]
        game_df['ResultTeam1'] = data[3]
        game_df['ResultTeam2'] = data[5]
        match_df = match_df.append(game_df)

    return match_df
    #match_df.to_clipboard()



    #data_df.iloc[7].values[0][0] -> maps
    #data_df.iloc[8].values[0][0] -> mapID
    #data_df.iloc[9].values[0][0] -> players

In [232]:
##MAIN
for j in range(0,1):   
    print("J: "+str(j))
        df_running = pd.read_csv("RunningTest.csv")
        total_matches = len(df_running['MatchID'].unique())
        n_runs = 2
        print("Stating at: "+ str(total_matches)+ "WITH " + str(n_runs) + " incs")
        
        runname = "RunningTest"

        #CAREFUL FETCHES PAGE (runtime=1)
        matchesLinks = get_matchLinks(total_matches) #goes to results page to fetch matchLinks
        #print(pd.DataFrame(matchesLinks))

        for i in range(0,n_runs): #len(matchesLinks)
            match_id = matchesLinks[i].split("/")[2]
            if len(df_running['MatchID'].where(df_running['MatchID'] == match_id).dropna()) != 0:
                print("Duplicate Skipping" + match_id)
            else:
                print("Fetching: " +str(match_id) + " I: "+ str(i))
                #CAREFUL FETCHES PAGE
                soup = get_page_data(matchesLinks[i]) #gets the soup for matchpage from match links
                gamelinks = get_game_links(soup) #gets the gamelinks from match links
                output = gather_match_stats(gamelinks)  #gets stats from gamelinks
                output['MatchID'] = match_id
                print(output)
                #output.to_csv(str(runname)+'.csv', mode='a', header=False,index=False)



J: 0
Stating at: 214WITH 10 incs
J: 1
Stating at: 214WITH 10 incs
J: 2
Stating at: 214WITH 10 incs
J: 3
Stating at: 214WITH 10 incs
J: 4
Stating at: 214WITH 10 incs
J: 5
Stating at: 214WITH 10 incs
J: 6
Stating at: 214WITH 10 incs
J: 7
Stating at: 214WITH 10 incs
J: 8
Stating at: 214WITH 10 incs
J: 9
Stating at: 214WITH 10 incs
J: 10
Stating at: 214WITH 10 incs
J: 11
Stating at: 214WITH 10 incs
J: 12
Stating at: 214WITH 10 incs
J: 13
Stating at: 214WITH 10 incs
J: 14
Stating at: 214WITH 10 incs
J: 15
Stating at: 214WITH 10 incs
J: 16
Stating at: 214WITH 10 incs
J: 17
Stating at: 214WITH 10 incs
J: 18
Stating at: 214WITH 10 incs
J: 19
Stating at: 214WITH 10 incs


In [223]:
df_running = pd.read_csv("RunningTest.csv")
len(df_running['MatchID'].where(df_running['MatchID'] == match_id).dropna())

0

In [229]:

df_running = pd.read_csv("RunningTest.csv")
total_matches = len(df_running['MatchID'].unique())

print("Stating at: "+ str(total_matches))
n_runs = 5
runname = "RunningTest"

#CAREFUL FETCHES PAGE (runtime=1)
matchesLinks = get_matchLinks(total_matches) #goes to results page to fetch matchLinks
#print(pd.DataFrame(matchesLinks))

for i in range(0,n_runs): #len(matchesLinks)
    match_id = matchesLinks[i].split("/")[2]
    if len(df_running['MatchID'].where(df_running['MatchID'] == match_id).dropna()) != 0:
        print("Duplicate Skipping" + match_id)
    else:
        print("Fetching: " +str(match_id) + "I: "+ str(i))
        #CAREFUL FETCHES PAGE
        soup = get_page_data(matchesLinks[i]) #gets the soup for matchpage from match links
        gamelinks = get_game_links(soup) #gets the gamelinks from match links
        output = gather_match_stats(gamelinks)  #gets stats from gamelinks
        output['MatchID'] = match_id

        #if(len(df_running['MatchID'].where(df_running['MatchID'] == match_id).dropna()) == 0):
        output.to_csv(str(runname)+'.csv', mode='a', header=False,index=False)

        #output.to_csv('match_'+str(match_id)+'.csv',index=False)
        #playersdf.to_csv('csplayers_nd_'+str(page)+'.csv',index=False)

Stating at: 9
Fetching: 2347024I: 0
Fetching: 2347025I: 1
Fetching: 2347738I: 2
Fetching: 2347017I: 3
Fetching: 2347016I: 4


In [186]:
matchesLinks_ids = []
for i in range(0,len(matchesLinks)):
    matchesLinks_ids.append(matchesLinks[i].split("/")[2])
print(len(matchesLinks[]))

54


In [204]:
df_running = pd.read_csv("run_Test_50.csv")
print(len(df_running['MatchID'].unique()))

48


for i in range(0,len(results)):
    print(results[i])
df = pd.DataFrame(result[0])
print(df)